In [1]:
import pandas as pd
p='artifacts/level3_llm/llm_parsed.parquet'
df=pd.read_parquet(p)
print("rows:", len(df))
print("valid adjustments:", int(df['llm_adjustment_valid'].sum()))
print("needs_review:", int(df['needs_review'].sum()))
print("llm_adj clamped stats:\n", df['llm_adjustment_clamped'].describe().to_string())
print("example evidence ids for first rows:\n", df[['transaction_id','llm_evidence_ids']].head(10).to_string(index=False))


rows: 1000
valid adjustments: 0
needs_review: 1000
llm_adj clamped stats:
 count    1000.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
example evidence ids for first rows:
 transaction_id llm_evidence_ids
       T100000               []
       T100001               []
       T100002               []
       T100003               []
       T100004               []
       T100005               []
       T100006               []
       T100007               []
       T100008               []
       T100009               []


In [2]:
import json
lines = open('artifacts/level3_llm/llm_raw.jsonl','r',encoding='utf-8').read().splitlines()
if not lines:
    print("no raw lines")
else:
    j = json.loads(lines[0])
    print("PROMPT (truncated):")
    print(j.get('prompt','')[:1000])
    print("\nRAW_OUTPUT:")
    print(j.get('raw_output','')[:2000])

PROMPT (truncated):

You are a strict fraud-triage assistant. Answer ONLY with a single valid JSON object that matches the schema exactly.

Schema:
{
  "transaction_id": "<string>",
  "llm_adjustment": <number>,       # additive delta to baseline score (range -0.10 .. +0.10). Will be clamped by caller to +/-0.0500
  "evidence_ids": ["E1","E2"],     # IDs referencing provided evidence items (must be subset of provided top_k_evidence ids)
  "explanation": "<string>",        # concise factual explanation based ONLY on provided evidence (<=250 chars)
  "confidence": <number>           # optional 0..1 numeric confidence
}

Rules:
- DO NOT output any text outside the JSON object (no markdown, no comments).
- Only reference evidence by id. evidence_ids MUST be a subset of the provided top_k_evidence ids.
- llm_adjustment must be a numeric additive delta; we will clamp it to +/- the configured max_delta.
- explanation must be concise and based solely on the evidence.
- Temperature must be 0 (d

In [2]:
import pandas as pd
p='artifacts/level3_llm_test/llm_parsed.parquet'
df=pd.read_parquet(p)
print("rows:", len(df))
print("valid adjustments:", int(df['llm_adjustment_valid'].sum()))
print("needs_review:", int(df['needs_review'].sum()))
print("llm_adj clamped stats:\n", df['llm_adjustment_clamped'].describe().to_string())
print(df[['transaction_id','llm_evidence_ids','llm_adjustment_clamped','needs_review']].head(10).to_string(index=False))


rows: 5
valid adjustments: 0
needs_review: 5
llm_adj clamped stats:
 count    5.0
mean     0.0
std      0.0
min      0.0
25%      0.0
50%      0.0
75%      0.0
max      0.0
transaction_id llm_evidence_ids  llm_adjustment_clamped  needs_review
       T100000               []                     0.0          True
       T100001               []                     0.0          True
       T100002               []                     0.0          True
       T100003               []                     0.0          True
       T100004               []                     0.0          True


In [3]:

import json
line = open('artifacts/level3_llm_test/llm_raw.jsonl','r',encoding='utf-8').readline()
print(json.dumps(json.loads(line), indent=2, ensure_ascii=False))

{
  "transaction_id": "T100000",
  "timestamp_utc": "2026-01-24T15:03:13.196882Z",
  "prompt": "\nYou are a strict fraud-triage assistant. Answer ONLY with a single valid JSON object that matches the schema exactly.\n\nSchema:\n{\n  \"transaction_id\": \"<string>\",\n  \"llm_adjustment\": <number>,       # additive delta to baseline score (range -0.10 .. +0.10). Will be clamped by caller to +/-0.0500\n  \"evidence_ids\": [\"E1\",\"E2\"],     # IDs referencing provided evidence items (must be subset of provided top_k_evidence ids)\n  \"explanation\": \"<string>\",        # concise factual explanation based ONLY on provided evidence (<=250 chars)\n  \"confidence\": <number>           # optional 0..1 numeric confidence\n}\n\nRules:\n- DO NOT output any text outside the JSON object (no markdown, no comments).\n- Only reference evidence by id. evidence_ids MUST be a subset of the provided top_k_evidence ids.\n- llm_adjustment must be a numeric additive delta; we will clamp it to +/- the con

In [4]:
import pandas as pd
res = pd.read_parquet('artifacts/results_stream.parquet')
llm = pd.read_parquet('artifacts/level3_llm_test/llm_parsed.parquet')

res['transaction_id'] = res['transaction_id'].astype(str)
llm['transaction_id'] = llm['transaction_id'].astype(str)

llm_sel = llm[['transaction_id','llm_adjustment_clamped','llm_adjustment_raw','llm_adjustment_valid','llm_evidence_ids','needs_review']].copy()
merged = res.merge(llm_sel, on='transaction_id', how='left')

merged['llm_adjustment_clamped'] = merged['llm_adjustment_clamped'].fillna(0.0)
merged['llm_adjustment_valid'] = merged['llm_adjustment_valid'].fillna(False)

merged.to_parquet('artifacts/results_stream_with_llm.parquet', index=False)
print("WROTE: artifacts/results_stream_with_llm.parquet")

WROTE: artifacts/results_stream_with_llm.parquet


In [6]:
import pandas as pd
before = pd.read_parquet('artifacts/results_stream.parquet')
after = pd.read_parquet('artifacts/results_stream_recombined_with_llm.parquet')
df = before.merge(after[['transaction_id','final_score']], on='transaction_id', suffixes=('_x', '_y'))
df['delta'] = df['final_score_y'] - df['final_score_x']
print("rows:", len(df))
print("rows changed:", int((df['delta']!=0).sum()), "fraction:", float((df['delta']!=0).mean()))
print("top positive deltas:\n", df.sort_values('delta',ascending=False).head(10).to_string(index=False))
print("top negative deltas:\n", df.sort_values('delta').head(10).to_string(index=False))

rows: 5000000
rows changed: 3857005 fraction: 0.771401
top positive deltas:
 transaction_id  baseline_score  anomaly_score  llm_adjustment  final_score_x decision  final_score_y  delta
      T5099999        0.043547       0.028381             0.0       0.043547    allow       0.043547    0.0
      T3736945        0.043547       0.022447             0.0       0.043547    allow       0.043547    0.0
      T3736899        0.043547       0.006114             0.0       0.043547    allow       0.043547    0.0
      T3736907        0.043547       0.019794             0.0       0.043547    allow       0.043547    0.0
      T1566711        0.043547       0.013271             0.0       0.043547    allow       0.043547    0.0
      T3736910        0.045162       0.000000             0.0       0.045162    allow       0.045162    0.0
      T3736912        0.043547       0.017218             0.0       0.043547    allow       0.043547    0.0
      T1566703        0.043547       0.017163             0

In [7]:
import pandas as pd, numpy as np
before = pd.read_parquet('artifacts/results_stream.parquet')  # original baseline file
after  = pd.read_parquet('artifacts/results_stream_recombined_with_llm.parquet')  # recombined output you produced

print("before columns:", before.columns.tolist()[:50])
print("after columns:", after.columns.tolist()[:50])

# Ensure transaction_id as str
before['transaction_id'] = before['transaction_id'].astype(str)
after['transaction_id']  = after['transaction_id'].astype(str)

# Merge to get baseline and final in one df
df = before[['transaction_id','baseline_score']].merge(after[['transaction_id','final_score','anomaly_score','llm_adjustment_clamped','llm_adjustment_valid']], on='transaction_id', how='left')

# Safety: fill missing columns if they don't exist
for c in ('final_score','anomaly_score','llm_adjustment_clamped','llm_adjustment_valid'):
    if c not in df.columns:
        df[c] = np.nan

# Compute delta
df['delta'] = df['final_score'].fillna(0.0) - df['baseline_score'].fillna(0.0)

print("total rows (merged):", len(df))
print("rows with final != baseline:", int((df['delta'] != 0).sum()), "fraction:", float((df['delta'] != 0).mean()))
print("delta stats:\n", df['delta'].describe().to_string())
# show extremes
print("\nTop positive deltas (10):")
print(df.sort_values('delta', ascending=False).head(10).to_string(index=False))
print("\nTop negative deltas (10):")
print(df.sort_values('delta').head(10).to_string(index=False))

before columns: ['transaction_id', 'baseline_score', 'anomaly_score', 'llm_adjustment', 'final_score', 'decision']
after columns: ['transaction_id', 'baseline_score', 'anomaly_score', 'llm_adjustment', 'final_score', 'decision', 'llm_adjustment_clamped', 'llm_adjustment_raw', 'llm_adjustment_valid', 'llm_evidence_ids', 'needs_review', 'topk_evidence_ids']
total rows (merged): 5000000
rows with final != baseline: 0 fraction: 0.0
delta stats:
 count    5000000.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0

Top positive deltas (10):
transaction_id  baseline_score  final_score  anomaly_score  llm_adjustment_clamped  llm_adjustment_valid  delta
       T100000        0.000000     0.000000       0.920105                     0.0                  True    0.0
      T3433331        0.043547     0.043547       0.935135                     0.0                  True    0.0
      T3433338        0.043547     0.04

In [8]:
import pandas as pd, numpy as np
before = pd.read_parquet('artifacts/results_stream.parquet')
after  = pd.read_parquet('artifacts/results_stream_recombined_with_llm.parquet')

before['transaction_id'] = before['transaction_id'].astype(str)
after['transaction_id']  = after['transaction_id'].astype(str)

# merge necessary cols
cols_after = ['transaction_id','final_score','anomaly_score','llm_adjustment_clamped','llm_adjustment_valid']
for c in cols_after[1:]:
    if c not in after.columns:
        after[c] = np.nan

df = before[['transaction_id','baseline_score']].merge(after[cols_after], on='transaction_id', how='left').fillna(0.0)

# intermediate after LLM (clamped to [0,1])
df['intermediate_llm'] = (df['baseline_score'] + df['llm_adjustment_clamped']).clip(0.0,1.0)
df['final_score'] = df['final_score'].clip(0.0,1.0)

tol = 1e-8
df['changed'] = (df['final_score'] - df['baseline_score']).abs() > tol
df['changed_by_llm_only'] = df['changed'] & (df['llm_adjustment_clamped'].abs() > tol) & (df['final_score'].sub(df['intermediate_llm']).abs() <= tol)
df['changed_by_anom'] = df['changed'] & (df['final_score'].sub(df['intermediate_llm']).abs() > tol)
df['llm_nonzero'] = df['llm_adjustment_clamped'].abs() > tol
df['anom_nonzero'] = df['anomaly_score'].abs() > tol

total = len(df)
n_changed = int(df['changed'].sum())
n_llm_only = int(df['changed_by_llm_only'].sum())
n_anom = int(df['changed_by_anom'].sum())
n_both = int(((df['llm_nonzero']) & (df['anom_nonzero']) & df['changed']).sum())

print("total rows:", total)
print("total changed:", n_changed, f"({n_changed/total:.3%})")
print("changed attributable to LLM only:", n_llm_only, f"({n_llm_only/total:.3%})")
print("changed where anomaly influenced final (may include LLM too):", n_anom, f"({n_anom/total:.3%})")
print("changed where both LLM nonzero and anomaly nonzero:", n_both, f"({n_both/total:.3%})")

# Sample rows for each bucket
print("\nSample changed_by_llm_only (up to 10):")
print(df[df['changed_by_llm_only']].head(10)[['transaction_id','baseline_score','llm_adjustment_clamped','intermediate_llm','final_score']].to_string(index=False))

print("\nSample changed_by_anom (up to 10):")
print(df[df['changed_by_anom']].head(10)[['transaction_id','baseline_score','anomaly_score','llm_adjustment_clamped','intermediate_llm','final_score']].to_string(index=False))

total rows: 5000000
total changed: 0 (0.000%)
changed attributable to LLM only: 0 (0.000%)
changed where anomaly influenced final (may include LLM too): 0 (0.000%)
changed where both LLM nonzero and anomaly nonzero: 0 (0.000%)

Sample changed_by_llm_only (up to 10):
Empty DataFrame
Columns: [transaction_id, baseline_score, llm_adjustment_clamped, intermediate_llm, final_score]
Index: []

Sample changed_by_anom (up to 10):
Empty DataFrame
Columns: [transaction_id, baseline_score, anomaly_score, llm_adjustment_clamped, intermediate_llm, final_score]
Index: []


In [1]:
import pandas as pd
p='artifacts/level3_llm/llm_parsed.parquet'
df=pd.read_parquet(p)
print("rows:", len(df))
print("llm_adjustment_valid:", int(df['llm_adjustment_valid'].sum()))
print("needs_review:", int(df['needs_review'].sum()))
print("non-zero clamped:", int((df['llm_adjustment_clamped']!=0).sum()))
print(df['llm_adjustment_clamped'].describe().to_string())

rows: 1000
llm_adjustment_valid: 0
needs_review: 1000
non-zero clamped: 0
count    1000.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0


In [2]:
import pandas as pd, json
p='artifacts/level3_llm/llm_parsed.parquet'
df=pd.read_parquet(p)
print("VALID examples:")
print(df[df['llm_adjustment_valid']==True].head(5)[['transaction_id','baseline_score','llm_adjustment_raw','llm_adjustment_clamped','llm_evidence_ids','explanation']].to_string(index=False))
print("\nNEEDS_REVIEW examples (show raw output):")
nr = df[df['needs_review']==True].head(5)
for tx in nr['transaction_id'].tolist():
    # find raw entry
    with open('artifacts/level3_llm/llm_raw.jsonl','r',encoding='utf-8') as fh:
        for line in fh:
            j=json.loads(line)
            if j.get('transaction_id')==tx:
                print("TX:",tx)
                print("PROMPT (truncated):", j.get('prompt','')[:400])
                print("RAW_OUTPUT (truncated):", j.get('raw_output','')[:400])
                print("----")
                break

VALID examples:
Empty DataFrame
Columns: [transaction_id, baseline_score, llm_adjustment_raw, llm_adjustment_clamped, llm_evidence_ids, explanation]
Index: []

NEEDS_REVIEW examples (show raw output):
TX: T100000
PROMPT (truncated): 
You are a strict fraud-triage assistant. Answer ONLY with a single valid JSON object that matches the schema exactly.

Schema:
{
  "transaction_id": "<string>",
  "llm_adjustment": <number>,       # additive delta to baseline score (range -0.10 .. +0.10). Will be clamped by caller to +/-0.0500
  "evidence_ids": ["E1","E2"],     # IDs referencing provided evidence items (must be subset of provided
RAW_OUTPUT (truncated): __OLLAMA_ERROR__ RuntimeError('ollama generate failed: rc=1, stderr=Error: unknown command "generate" for "ollama"\n')
----
TX: T100001
PROMPT (truncated): 
You are a strict fraud-triage assistant. Answer ONLY with a single valid JSON object that matches the schema exactly.

Schema:
{
  "transaction_id": "<string>",
  "llm_adjustment": <num

In [3]:
import json
line = open('artifacts/level3_llm_test/llm_raw.jsonl','r',encoding='utf-8').readline()
print(json.dumps(json.loads(line), indent=2))

{
  "transaction_id": "T100000",
  "timestamp_utc": "2026-01-24T15:03:13.196882Z",
  "prompt": "\nYou are a strict fraud-triage assistant. Answer ONLY with a single valid JSON object that matches the schema exactly.\n\nSchema:\n{\n  \"transaction_id\": \"<string>\",\n  \"llm_adjustment\": <number>,       # additive delta to baseline score (range -0.10 .. +0.10). Will be clamped by caller to +/-0.0500\n  \"evidence_ids\": [\"E1\",\"E2\"],     # IDs referencing provided evidence items (must be subset of provided top_k_evidence ids)\n  \"explanation\": \"<string>\",        # concise factual explanation based ONLY on provided evidence (<=250 chars)\n  \"confidence\": <number>           # optional 0..1 numeric confidence\n}\n\nRules:\n- DO NOT output any text outside the JSON object (no markdown, no comments).\n- Only reference evidence by id. evidence_ids MUST be a subset of the provided top_k_evidence ids.\n- llm_adjustment must be a numeric additive delta; we will clamp it to +/- the con

In [2]:
import json, pathlib
p=pathlib.Path('artifacts/level3_llm_test/llm_raw.jsonl')
print(p.exists())
if p.exists():
    print(json.dumps(json.loads(p.read_text().splitlines()[0]), indent=2))

True
{
  "transaction_id": "T100000",
  "timestamp_utc": "2026-01-24T15:03:13.196882Z",
  "prompt": "\nYou are a strict fraud-triage assistant. Answer ONLY with a single valid JSON object that matches the schema exactly.\n\nSchema:\n{\n  \"transaction_id\": \"<string>\",\n  \"llm_adjustment\": <number>,       # additive delta to baseline score (range -0.10 .. +0.10). Will be clamped by caller to +/-0.0500\n  \"evidence_ids\": [\"E1\",\"E2\"],     # IDs referencing provided evidence items (must be subset of provided top_k_evidence ids)\n  \"explanation\": \"<string>\",        # concise factual explanation based ONLY on provided evidence (<=250 chars)\n  \"confidence\": <number>           # optional 0..1 numeric confidence\n}\n\nRules:\n- DO NOT output any text outside the JSON object (no markdown, no comments).\n- Only reference evidence by id. evidence_ids MUST be a subset of the provided top_k_evidence ids.\n- llm_adjustment must be a numeric additive delta; we will clamp it to +/- th